## **Zeros de transmissão**

Na descrição entrada-saída (função de transferência) de um sistema, definimos zeros de uma forma bem matemática: são as raízes do numerador.

No entanto, os zeros possuem uma definição um pouco mais física. São chamados zeros os modos (exponenciais) que, se colocados na entrada do sistema, produzem uma saída identicamente nula. Isso permite estabelecer as expressões que definem os zeros do sistema em função das matrizes de espaço de estados.

$$
\begin{align}
    \text{Sistema: } u(t) &\longmapsto y(t)\\
    -a \text{ é um zero do sistema } \Leftrightarrow e^{-at} &\longmapsto 0
\end{align}
$$


Para achar os zeros, basta resolver a equação polinomial:
$$
\begin{align*}
\det\,\left[\begin{array}{cc}s\mathbf{I-A} & -\mathbf{B}\\ \mathbf{C} & D\end{array}\right]=0 
\end{align*}
$$

<p align="center">
<img src="Ex7.13.svg" width="80%">
</p>

**Solução**: Precisamos montar a matriz. No caso, já calculamos $s\mathbf{I-F}$ no exemplo anterior, então basta concatenar as demais:
$$
\begin{align*}
    s\mathbf{I-F} &= \left[\begin{array}{cc}s+7 & 12\\-1 & s\end{array}\right]\\
    \Rightarrow \left[\begin{array}{ccc}s\mathbf{I-F} & -\mathbf{G}\\ \mathbf{H} & J\end{array}\right]&=
    \left[\begin{array}{ccc}s+7 & 12 & -1\\-1 & s & 0\\ 1 & 2 & 0 \end{array}\right]
\end{align*}
$$

Basta agora calcular o determinante.
$$
\begin{align*}
    \det \left[\begin{array}{ccc}s+7 & 12 & 1\\-1 & s & 0\\ 1 & 2 & 0 \end{array}\right] &= 
    \det \left[\begin{array}{cc}-1 & s \\ 1 & 2 \end{array}\right] = -2-s 
\end{align*}
$$

Igualando a zero, temos:
$$
\begin{align*}
    -2-s =0 \Rightarrow s = -2
\end{align*}
$$

Podemos resolver isso com o SymPy

In [1]:
# Imports
import sympy as sp
from IPython.display import display, Math

In [5]:
# Define os símbolos e constrói o problema usando matrizes em bloco
s = sp.symbols('s')
A = sp.Matrix([[-7,-12],[1,0]])
B = sp.Matrix([[1],[0]])
C = sp.Matrix([[1,2]])
D = sp.Matrix([[0]])
M = sp.BlockMatrix([[s*sp.eye(2)-A, B],[C,D]])
display(Math(r'\mathbf{M} = ' + sp.latex(M)))

<IPython.core.display.Math object>

In [8]:
# Resolve o determinante
determinante = sp.det(M)
display(Math(r'\det(M) = ' + sp.latex(determinante)))
determinante = determinante.simplify()
display(Math(r'=' + sp.latex(determinante)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [13]:
# Iguala a zero e acha a raiz
sol = sp.solve(sp.Eq(determinante,0))
display(Math('s = ' + sp.latex(sol[0])))

<IPython.core.display.Math object>

Alternativamente, a função "zero()" do módulo "control" é capaz de determinar os zeros a partir da representação do sistema

In [16]:
# Imports
import numpy as np
import control as ct
import scipy.signal as sig
from escrever import *
esc = escrever()

In [20]:
# Declara o sistema
F = np.array([[-7,-12],[1,0]])
G = np.array([[1],[0]])
H = np.array([[1,2]])
J = np.array([[0]])

# Definindo o sistema como espaço de estados
sys = ct.ss(F,G,H,J)
esc.sist(F,G,H,J)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [21]:
# Calcula os zeros pela função "zero()"
print('Zeros: ', ct.zero(sys))

Zeros:  [-2.+0.j]


## **Exemplo**

<p align="center">
<img src="Ex7.14.svg" width="80%">
</p>

Esse é um problema de ordem elevada, então vamos resolver com Python

In [22]:
# Declara as matrizes e o sistema
F = np.array([[0,2,0,0,0],[-0.1,-0.35,0.1,0.1,0.75],[0,0,0,2,0],[.4,.4,-.4,-1.4,0],[0,-.03,0,0,-1]])
G = np.array([[0],[0],[0],[0],[1]])
H2 = np.array([[0,0,1,0,0]])
J = np.array([[0]])

sys = ct.ss(F,G,H2,J)
esc.sist(F,G,H2,J)


<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [23]:
# Polos e zeros
print('Pólos: ')
print(ct.pole(sys).round(decimals=4))
print('')
print('Zeros: ')
print(ct.zero(sys))

Pólos: 
[-0.6371+0.6669j -0.6371-0.6669j  0.    +0.j     -0.5075+0.j
 -0.9683+0.j    ]

Zeros: 
[-2.+0.j]


In [24]:
# Amortecimentos e frequências naturais
tab=ct.damp(sys)

_____Eigenvalue______ Damping___ Frequency_
   -0.6371   +0.6669j     0.6908     0.9223
   -0.6371   -0.6669j     0.6908     0.9223
 2.265e-16                     1 -2.265e-16
   -0.5075                     1     0.5075
   -0.9683                     1     0.9683


In [25]:
# Função de transferência
FT = ct.tf(sys)
num,den = ct.tfdata(sys)
num = num[0]
den = den[0]
FT

TransferFunction(array([1.77635684e-15, 3.10862447e-15, 1.55431223e-15, 6.00000000e-01,
       1.20000000e+00]), array([ 1.00000000e+00,  2.75000000e+00,  3.22250000e+00,  1.88150000e+00,
        4.18000000e-01, -9.46900958e-17]))

Esta função possui coeficientes que tipicamente chamamos de "mal condicinados". Isso ocorre devido a erros de truncamento no computador. 

Observe que os coeficientes "estranhos", comparados aos outros são muito pequenos (potência de 10 muito pequena.)

In [26]:
# Pólos calculados pela função de transferência
ct.pole(FT).round(decimals=4)

C:\Users\rafael\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\signal\_filter_design.py:1746: BadCoefficients: Badly conditioned filter coefficients (numerator): the results may be meaningless
  warnings.warn("Badly conditioned filter coefficients (numerator): the "


array([-0.6371+0.6669j, -0.6371-0.6669j, -0.9683+0.j    , -0.5075+0.j    ,
        0.    +0.j    ])

Perceba que o Python emite um aviso de condicionamento numérico. Apesar disso, os pólos concordam com aqueles calculados pelo espaço de estados. Repare que o cálculo feito pelas matrizes não resultou em mal condicionamento. 

In [27]:
# Zeros calculados pela FT
ct.zeros(FT).round(decimals=2)

array([ 3.482128e+04+60312.08j,  3.482128e+04-60312.08j,
       -6.964231e+04    +0.j  , -2.000000e+00    +0.j  ])

Percebemos que a diferença é grande em relação às matrizes, que resultou em apenas um zero. Pela FT, obtivemos seis zeros, alguns deles de valor elevado e no semi-plano direito

Como mencionado, isso é apenas um problema numérico e pode ser resolvido meramente arredondando os coeficientes da função de transferência. Uma forma de fazer é conforme o código a seguir:

In [28]:
# Redefine a função de transferência com arredondamentos
FT2 = ct.tf(num[0].round(decimals=4),den[0].round(decimals=4))
FT2

TransferFunction(array([0.6, 1.2]), array([ 1.    ,  2.75  ,  3.2225,  1.8815,  0.418 , -0.    ]))

In [29]:
# Recalcula os pólos e zeros pela função de transferência
print("Pólos recalculados: ")
print(ct.pole(FT2).round(decimals=4))
print('')
print("Zeros recalculados: ")
print(ct.zeros(FT2).round(decimals=4))

Pólos recalculados: 
[-0.6371+0.6669j -0.6371-0.6669j -0.9683+0.j     -0.5075+0.j
  0.    +0.j    ]

Zeros recalculados: 
[-2.+0.j]


Para finalizar, você sempre deve atentar à posição dos zeros em relação aos pólos do sistema, sobretudo os dominantes de malha fechada, no caso de projeto de controle.

Ao terminar um projeto, **sempre** imprima um relatório final de pólos e zeros para antecipar o que está acontecendo com a resposta simulada. Plotar uma resposta ao degrau sem essa análise torna sua resposta final incompleta.